In [ ]:
import jsonlines
import json
import torch
from transformers import AutoModel, AutoTokenizer

In [ ]:
# !pip install jsonlines
# !pip install transformers

In [ ]:
with open('/content/train.jsonl', 'r') as json_file:
    json_list = list(json_file)

train_dbs, test_dbs, dev_dbs = [], [], []

for json_str in json_list:
    result = json.loads(json_str)
    train_dbs.append(result)
    # print(f"result: {result}")
    # print(isinstance(result, dict))

In [ ]:
MODEL_NAME = "t5-base"

# We need to create the model and tokenizer
model = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
# !pip install datasets

ERROR: Could not find a version that satisfies the requirement datasets.Dataset (from versions: none)
ERROR: No matching distribution found for datasets.Dataset


In [ ]:


from datasets import Dataset, DatasetDict
import pandas as pd

# perfectIR
# train_dataset = []
# for db in train_dbs:
#   for query in db['queries']:
#     query_text = query['query']
#     query_facts = query['facts']

#     query_facts = [db['metadata']['raw'][fact[0]]['parse_targets'] for fact in query_facts]
#     query_facts = [fact for sub_fact in query_facts for fact in sub_fact ]

#     text = ''

#     for fact in query_facts:
#       text += fact + ' '

#     text = text[:-1]
#     train_dataset.append([query_text, text])

# wholeDB
train_dataset = []
for db in train_dbs:

  db_facts = [db['metadata']['raw'][:]['parse_targets']]
  db_facts = [fact for sub_fact in db_facts for fact in sub_fact ]

  query_facts = ''

  for fact in db_facts:
    query_facts += fact + ' '

  query_facts = query_facts[:-1]

  for query in db['queries']:
    query_text = query['query']
    query_facts = query['facts']

    train_dataset.append([query_text, query_facts])

df = pd.DataFrame(train_dataset, columns = ['query', 'facts'])
td = Dataset.from_pandas(df)
dataset_dict = {"train": td,
           "test": td,
           "unsupervised": td}

d = DatasetDict(dataset_dict)
d

In [ ]:
# encoding = tokenizer.encode_plus(train_dataset[0], add_special_tokens = True,    truncation = True, padding = "max_length", return_attention_mask = True, return_tensors = "pt")
def tokenize_function(examples):
  print(examples)
  return tokenizer(examples["query"],examples["facts"], padding="max_length", truncation=True)
tokenized_datasets = d.map(tokenize_function, batched=True)

In [ ]:
tokenized_datasets

In [ ]:
# tokenized_datasets.get("train")["input_ids"][0]
for x in tokenized_datasets.get("train")["input_ids"][0]:
    print(tokenizer.decode(x))

In [ ]:
# from transformers import pipeline
# qna = pipeline("question-answering", model="t5-base", tokenizer="t5-base", framework="tf")

# from transformers import default_data_collator

# data_collator = default_data_collator
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    num_train_epochs=3,
    weight_decay=0.01,
    output_dir="/content/"
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets.get("train"),
    eval_dataset=tokenized_datasets.get("test")
)

trainer.train()